In [1]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

import shutil

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime as dt

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_SQL_a_DataFrame
import funcion_parametrizacion_variables
import funcion_migracionReporte_GoogleDrive
import funcion_exportacion_pdf_seriemapas
import funcion_copia_googledrive

In [2]:

fecha_actual = dt.now()
fecha_estandarizada = str(fecha_actual.strftime("%Y%m%d"))

FC_TERRENOS_UNIFICADOS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_POR_HITO"
FC_BD_INDICADOR = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_11_Indicador_NPN_EntregaProdFinales\Base_Datos\2_11_NPNProductosFinales.gdb"

# TODO: Actualización del Reporte de Seguimiento
DIRECTORIO_SALIDA_XLSX = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_2_Rendimientos_Reconocimiento"
NOMBRE_XLSX = fecha_estandarizada + "_RendimientoEquipoReconocimiento_BCGS.xlsx"
RUTA_SALIDA_XLSX = os.path.join(DIRECTORIO_SALIDA_XLSX, NOMBRE_XLSX)
NOMBRE_HOJA = 'NPN_En_EntregaProductoFinal'

In [3]:

# TODO: Llamado a la consulta 
resultados = funcion_parametrizacion_variables.parametrizacion_variables()
consulta_tRendimientosActualizacion = resultados[2]
renombrar_actividades = resultados[4]

# TODO: Generación de DF a partir de consulta
df_tRendimientosActualizacion = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_tRendimientosActualizacion)

for columna in df_tRendimientosActualizacion.columns:
    if columna == 'actividad_actual_tramite':
        for llave, valor in renombrar_actividades.items():
            df_tRendimientosActualizacion.loc[df_tRendimientosActualizacion[columna]==llave, columna] = valor

Inicio de la conexión
Conexión exitosa ...


In [4]:

# TODO: Preparación y selección de NPN en estado de actividad 16.Realizando Entrega de Productos Finales
df_tRendimientosActualizacion_ProdFinales = df_tRendimientosActualizacion[df_tRendimientosActualizacion['actividad_actual_tramite'] == '16.Realizando Entrega de Productos Finales'][['municipio','npn','pversion','radicado','ficha','tipo_terreno']]
df_tRendimientosActualizacion_ProdFinales = (df_tRendimientosActualizacion_ProdFinales.rename(columns={'pversion':'radicado_padre','radicado':'radicado_hijo'})).astype({'radicado_padre':str, 'radicado_hijo':str})
df_tRendimientosActualizacion_ProdFinales['radicado_padre'] = df_tRendimientosActualizacion_ProdFinales['radicado_padre'].astype(float).astype(int).astype(str)

In [5]:

# TODO: Acceso a la capa Terrenos mediante DF
df_terrenosHito = pd.DataFrame.spatial.from_featureclass(FC_TERRENOS_UNIFICADOS)
df_terrenosHito = df_terrenosHito[['codigo','codigo_anterior','fuente','area_ha_cmt12','meta_hito','id_ui','SHAPE']]

# TODO: Espacialización de NPN Tipo 16
df_Unificacion_ProdFinales_Terrenos = df_tRendimientosActualizacion_ProdFinales.merge(right=df_terrenosHito, left_on='npn', right_on='codigo', how='left')
df_Unificacion_ProdFinales_Terrenos.spatial.to_featureclass(location=os.path.join(FC_BD_INDICADOR,'terrenos_prodFinales'))
print("Se crea la capa geográfica del indicador")

Se crea la capa geográfica del indicador


In [6]:

# TODO: Se actualiza el reporte de rendimientos
with pd.ExcelWriter(RUTA_SALIDA_XLSX, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_tRendimientosActualizacion_ProdFinales.to_excel(writer, sheet_name=NOMBRE_HOJA, index=False)

In [7]:
RUTA_PDF = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_11_Indicador_NPN_EntregaProdFinales\Salidas_Graficas"
nombre_layout = 'Seguimiento NPN Entrega Prod. Finales'

# TODO: Generación de Salidas Gráficas
funcion_exportacion_pdf_seriemapas.exportacion_salida_grafica(nombre_layout,RUTA_PDF)

Nombre Layout: Seguimiento NPN Entrega Prod. Finales
Se exporta a PDF Seguimiento NPN Entrega Prod. Finales


In [8]:
dLocal = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_11_Indicador_NPN_EntregaProdFinales"
dGoogleDrive = r"G:\Mi unidad\Equipo_Consolidacion\Hitos\E2_Informes_Id_FisicoJuridica\2_2_11_Indicador_NPN_EntregaProdFinales"

funcion_copia_googledrive.migracion_googledrive(dLocal, dGoogleDrive)
print("Se migra los archivos del Indicador")

Se copia BD 2_11_NPNProductosFinales.gdb
Se copia archivo PDF 20240905_María La Baja.pdf
Se copia archivo PDF 20240905_Repelón.pdf
Se copia archivo PDF 20240906_María La Baja.pdf
Se copia archivo PDF 20240906_Repelón.pdf
Se migra los archivos del Indicador


<h3>Migración a Google Drive</h3>

In [9]:
funcion_migracionReporte_GoogleDrive.reporte_a_googleDrive()
print("Se migra a Google Drive")

Se migra a Google Drive
